In [1]:
import ECGDataset 
import Models 
import Net
from train_test_validat import *
from self_attention import *
import matplotlib.pyplot as plt
import ecg_plot
import cam
import ECGplot
import ECGHandle
import res1d
import torch
import torch.utils.data as Data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import random

import pandas as pd
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc,precision_recall_curve

import time
import math
import os
import gc
from torch.utils.tensorboard import SummaryWriter


def seed_torch(seed=2023):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
	torch.backends.cudnn.benchmark = False 
	torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.enabled = False

def filter_diagnose(df_input,remove_diagnose = ''): 
    df_filter = df_input.copy()
    if(remove_diagnose):
        fitler_ID_list = df_filter[(df_filter['concat'].str.contains(remove_diagnose) == True)]['ID'].tolist()
        fitler_index = df_filter[[True if i in fitler_ID_list else False for i in df_filter['ID']]].index #选取出所有含有该ID的样本
        df_remove = df_filter.loc[(fitler_index)]
        no_fitler_index = df_filter[[False if i in fitler_ID_list else True for i in df_filter['ID']]].index #选取出所有不含有该ID的样本
        df_filter = df_filter.loc[(no_fitler_index)]
        
        print('\n')
        print("{:^10} {:^10} {:^20}".format('  ','orginal','remove diagnose' + remove_diagnose))
        print("{:^10} {:^10} {:^20}".format('nums',len(df_input),len(df_filter)))
        print("{:^10} {:^10} {:^20}".format('  ','HTN','NHTN'))
        print("{:^10} {:^10} {:^20}".format('nums',len(df_filter[(df_filter['label']==1)]),len(df_filter[(df_filter['label']==0)])))
        print("{:^10} {:^10} {:^20}".format('  ','remove HTN','remove NHTN'))
        print("{:^10} {:^10} {:^20}".format('nums',len(df_remove[(df_remove['label']==1)]),len(df_remove[(df_remove['label']==0)])))
        return df_filter

In [2]:
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = "cpu"
seed_torch(2023)
plt.rcParams['font.sans-serif'] = ['SimHei'] #或者把"SimHei"换为"KaiTi"

supplement_diagnose = pd.read_csv('./补充诊断.csv',encoding='utf-8-sig')
supplement_diagnose['ID'] = supplement_diagnose['ID'].astype(str)
# 使用groupby方法按照ID分组，然后使用agg方法将data列拼接在一起
supplement_diagnose = supplement_diagnose.groupby('ID')['住院所有诊断'].agg(lambda x: ','.join(x.astype(str))).reset_index()
# 使用str.contains方法筛选data列中包含‘高血压’的行
filtered_df = supplement_diagnose[supplement_diagnose['住院所有诊断'].str.contains('高血压')]
# 将筛选后的ID列转换为list
id_list = filtered_df['ID'].astype(str).tolist()

In [4]:
supplement_diagnose = pd.read_csv('./补充诊断.csv',encoding='utf-8-sig')
supplement_diagnose['ID'] = supplement_diagnose['ID'].astype(str)
# 使用groupby方法按照ID分组，然后使用agg方法将data列拼接在一起
supplement_diagnose = supplement_diagnose.groupby('ID')['住院所有诊断'].agg(lambda x: ','.join(x.astype(str))).reset_index()
# 使用str.contains方法筛选data列中包含‘高血压’的行
data_root = '/workspace/data/Preprocess_HTN/datas_/'
ALL_data = pd.read_csv(data_root+'/All_data_handled_ID_range_age_IDimputate.csv',low_memory=False)
# ALL_data = ECGHandle.correct_age(ALL_data)
ALL_data = ALL_data.rename(columns={'住院号':'ID','年龄':'age','性别':'gender','姓名':'name'}) 
ALL_data = ALL_data[(ALL_data['申请部门'].str.contains('重症') == False)]
ALL_data = ALL_data[(~ALL_data['ID'].isnull())] #ID NULL
ALL_data = ALL_data[(~ALL_data['gender'].isnull())] #ID NULL
ALL_data = ALL_data[(ALL_data['Q']<1)&(~ALL_data['Q'].isnull())]#q_sum<qc_threshold
ALL_data = ALL_data[((ALL_data['age'].apply(int))>17) ]# 选年龄

ALL_data['诊断'] = ALL_data['诊断'].fillna(value='')
ALL_data['临床诊断'] = ALL_data['临床诊断'].fillna(value='')

#添加补充诊断
ALL_data = pd.merge(ALL_data,supplement_diagnose,how='inner',on='ID').reset_index()
# 使用groupby和agg函数将具有相同'ID'的行的data、data1和data2列拼接在一起
# 使用lambda函数和join方法将每个分组的值用逗号分隔
# 保留所有的行和列，使用merge方法将拼接后的结果与原始数据框合并
# 重置索引并重命名新的列为'concat'
df_concat = ALL_data.groupby('ID')[[ '诊断','住院所有诊断', '临床诊断',]].agg(lambda x: ','.join(x)).reset_index()
df_concat['concat'] = df_concat.apply(lambda x: ','.join([x['诊断'], x['住院所有诊断'], x['临床诊断']]), axis=1)
df_concat.drop(['诊断','住院所有诊断', '临床诊断'], axis=1, inplace=True) 
df_merge = ALL_data.merge(df_concat, on='ID', how='left') 

df_merge.loc[(df_merge['concat'].str.contains('高血压')==True),'label']= 1 # concat diagnose含有高血压的label为1
df_merge.loc[~(df_merge['label']==1),'label']= 0 #diagnose不含有高血压的label为0

# Replace '男' with 1 and '女' with 2 in the 'gender' column of df
df_merge['gender'].replace({'男': 1, '女': 2}, inplace=True)
df_merge['label'] = pd.to_numeric(df_merge['label'],errors='coerce', downcast='integer') #把label（diagnose）改成数值型
df_merge['age'] = pd.to_numeric(df_merge['age'],errors='coerce', downcast='integer') #把label（diagnose）改成数值型
df_merge['gender'] = pd.to_numeric(df_merge['gender'],errors='coerce', downcast='integer') #把label（diagnose）改成数值型

df_merge.drop_duplicates(subset=['ID'],keep='last')#删除重复的

print("{:^10} {:^10} {:^20}".format('  ','HTN','NHTN'))
print("{:^10} {:^10} {:^20}".format('nums',len(df_merge[(df_merge['label']==1)]),len(df_merge[(df_merge['label']==0)])))

# df_merge = ECGHandle.filter_departmentORlabel(df_merge,'外')

df_merge = filter_diagnose(df_merge,'起搏')
df_merge = filter_diagnose(df_merge,'除颤')
df_merge = filter_diagnose(df_merge,'电解质')
df_merge = filter_diagnose(df_merge,'钙血')
df_merge = filter_diagnose(df_merge,'钾血')
df_merge = filter_diagnose(df_merge,'镁血')

df_merge = filter_diagnose(df_merge,'瓣膜')
df_merge = filter_diagnose(df_merge,'脉瓣')
df_merge = filter_diagnose(df_merge,'尖瓣')
df_merge = filter_diagnose(df_merge,'肾上腺')

ALL_data_buffer = df_merge.copy()
ALL_data_buffer = ALL_data_buffer.sample(frac=1).reset_index(drop=True) #打乱顺序
test_df = ALL_data_buffer[ALL_data_buffer['year']==22].reset_index(drop=True)
tv_df = ALL_data_buffer[ALL_data_buffer['year']!=22].reset_index(drop=True)

              HTN             NHTN        
   nums      14142           33467        


            orginal    remove diagnose起搏  
   nums      47609           47003        
              HTN             NHTN        
   nums      13765           33238        
           remove HTN     remove NHTN     
   nums       377             229         


            orginal    remove diagnose除颤  
   nums      47003           47003        
              HTN             NHTN        
   nums      13765           33238        
           remove HTN     remove NHTN     
   nums        0               0          


            orginal    remove diagnose电解质 
   nums      47003           45213        
              HTN             NHTN        
   nums      12943           32270        
           remove HTN     remove NHTN     
   nums       822             968         


            orginal    remove diagnose钙血  
   nums      45213           44922        
              HTN             NHTN        
   

In [4]:
import numpy as np
from scipy import signal

def downsample_ecg(ecg_data):
    # 500Hz to 100Hz
    resampled_ecg = signal.resample(ecg_data, num=1000, axis=-1)
    return resampled_ecg

# Example usage
ecg_data = np.random.rand(10, 12, 5000) # Example data
resampled_ecg = downsample_ecg(ecg_data)